In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import json
import glob
import tensorflow as tf
import time

#Add running from cmd.

SEQ_LEN = 25


2021-07-22 23:40:09.140191: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-22 23:40:09.140241: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def get_urls(query, pages=1, get_top_only=True):
    urls = []
    for page in range(1, pages+1):
        url = f'https://www.ultimate-guitar.com/search.php?page={page}&title={query.replace(" ", "%20")}&type=200'
        r = requests.get(url)
        if r.status_code == 404: return urls #finished iterating over all pages.
        soup = BeautifulSoup(r.text)
        j = soup.find("div", {"class": "js-store"})['data-content']
        tabs = json.loads(j)['store']['page']['data']['results']

        song_id = 0
        for tab in tabs:
            if 'marketing_type' in tab: continue #ignore paid tabs.
            if not get_top_only: urls.append(tab['tab_url']); continue
            if song_id == 0: song_id, rating, url = tab['song_id'], tab['rating'], tab['tab_url']
                
            if song_id != tab['song_id']:
                if rating > 3: #don't append bad tabs.
                    urls.append(url)
                    print(f'Best: {rating} - {url}')
                song_id = tab['song_id']
                rating = tab['rating']
                url = tab['tab_url']

            if tab['rating'] > rating:
                rating = tab['rating']
                url = tab['tab_url']
                
    return urls

def get_urls_top(): #gets top 100 tabs.
    urls = []
    url = 'https://www.ultimate-guitar.com/top/tabs?order=hitsdailygroup_desc&type=tab'
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    j = soup.find("div", {"class": "js-store"})['data-content']
    tabs = json.loads(j)['store']['page']['data']['tabs']#['results']
    print(tabs)

    for tab in tabs:
        if 'marketing_type' in tab: continue #paid tabs.
        urls.append(tab['tab_url'])
    return urls

#urls = get_urls('pink floyd', 10)
#urls = get_urls_top(2)
urls = []
urls

[]

In [3]:

def get_tab(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    j = soup.body.find_all("div")[2]['data-content']
    text = json.loads(j)['store']['page']['data']["tab_view"]['wiki_tab']['content']
    return text

for url in urls:
    time.sleep(0.5) #spamming requests blocks you.
    text = get_tab(url)
    name = url.split('/')[-1]
    with open(f"tabs/{name}", 'w') as f:
        f.write(text)
    print(f'{name} downloaded.')

In [55]:
import struct
#Seperates tab from other notation
def seperate_tabs(text):
    tabs = []
    count = 0
    text = text.replace('[tab]', '').replace('[/tab]', '').replace(' ', '')
    text_split = text.split('\n')
    for i, l in enumerate(text_split):
        if len(l) < 10: count = 0; continue
        if l[0] == '|' and l[1] in '-123456789(': #if string is not specified, label the correct string (assuming EADGBe)
            if count == 6: return #Tabs are for 7+ strings. Disregard the tab.
            string = 'eBGDAE'[count]
            count += 1
            l = string + l
            
        else: count = 0
        
        if l[0].lower() in 'ebgda' and l[1] == '|':
            if l[0] == 'E' and i>0 and len(text_split[i-1])>0 and text_split[i-1][0] not in 'A|': l = 'e' + l[1:] #sometimes people label high e as E. lowercase it.
            stripped_line = l.replace('|', '').replace('\r', '').replace(' ', '')
            tabs.append(stripped_line)
    return tabs


def pad_tabs(tabs):
    longest = max([len(l) for l in tabs])
    tabs = [l + '-' * (longest-len(l)) for l in tabs]
    return tabs


#Concatenates tabs by common string.
def join_tabs(tabs):
    strings = {}
    for l in tabs:
        if l[0] not in strings:
            strings[l[0]] = ''
            continue
        strings[l[0]] += l[1:]
    tab_list = list(strings.values())
    tab_list = pad_tabs(tab_list)

    return tab_list

tab_dict = {'-': 1, '1': 2, '2': 3, '0': 4, '7': 5, '5': 6, '4': 7, '3': 8, '9': 9, 'x': 10, '6': 11, 'b': 12, '(': 13, ')': 14, '8': 15, '/': 16, '~': 17, 'p': 18, 'h': 19, '=': 20, '\\': 21, 's': 22, "'": 23, 'r': 24}
seq_dict = {0: 'O', 1: '-', 2: '1', 3: '2', 4: '0', 5: '7', 6: '5', 7: '4', 8: '3', 9: '9', 10: 'x', 11: '6', 12: 'b', 13: '(', 14: ')', 15: '8', 16: '/', 17: '~', 18: 'p', 19: 'h', 20: '=', 21: '\\', 22: 's', 23: "'", 24: 'r'}


text_to_seq = lambda text: [tab_dict.get(s.lower()) or 0 for s in text] #0 is out of vocab.
seq_to_text = lambda seq: [seq_dict[n] for n in seq]

def tab_to_seq(tab):
    for i, s in enumerate(tab):
        tab[i] = text_to_seq(s)
    return tab


def write_tab_binary(tab_list): #custom file format: byte = tab-char index, 6bytes = timestep.
    with open('data', 'ab') as f:
        for i in range(len(tab_list[0])):
            binary = struct.pack('6B', tab_list[0][i], tab_list[1][i], tab_list[2][i], tab_list[3][i], tab_list[4][i], tab_list[5][i])
            f.write(binary)
        

tab_lists = []
for filename in glob.glob('tabs/*'):
    with open(filename, 'r') as f:
        text = f.read()
    tabs = seperate_tabs(text)
    if not tabs: continue
    tab_list = join_tabs(tabs)
    if len(tab_list) != 6 or len(tab_list[0]) < SEQ_LEN+1: continue #if string_num isn't 6 or tab length is shorter than 1 sequence__lem, skip tab
    tab_list = tab_to_seq(tab_list)
    write_tab_binary(tab_list)
    #tab_lists.append(tab_list)
    
    
with open('data', 'rb') as f:
    #for i in range(60):
    d = f.read(6*SEQ_LEN)
    print(struct.unpack(f'{6*SEQ_LEN}B', d)t)
    #print(struct.unpack('BBBBBB', d))






# chars = sorted(list(set(concatenated))) #manually creating a dict is better.
# char_dict = dict((c, i) for i, c in enumerate(chars))
# reversed_char_dict = dict((i, c) for i, c in enumerate(chars))
# tokenize_list = lambda s: [char_dict[c] for c in s]
# detokenize_list = lambda s: [reversed_char_dict[n] for n in s]

def one_hot(arr):
    encoded = np.zeros((len(arr), len(char_dict)), dtype='bool') #uint8
    for i, n in enumerate(arr):
        encoded[i][n] = 1
    return encoded
    
def create_seqs(arr, seq_len=SEQ_LEN):
    seqs = []
    for i in range(len(arr) - seq_len):
        seqs.append(arr[i:i+seq_len])
    return np.array(seqs)
    
#one-hot all the tabs.

for i, tab in enumerate(tab_lists):
    encoded_list = []
    for s in tab:
        encoded_list.append(one_hot(tokenize_list(s)))

    #reshape 6/seq/chars -> seq/6/chars
    encoded_list = np.array(encoded_list).swapaxes(0,1)
    if i == 0:
        seqs = create_seqs(encoded_list)
        print(seqs.shape)
        continue
    seqs = np.concatenate([seqs, create_seqs(encoded_list)])
#     seqs = create_seqs(encoded_list)
#     with open('numpy_data.npy', 'ab') as f:
#         np.save(f, seqs)
#     print(seqs.shape)
    print(seqs.shape)

print(seqs.shape)
#LSTM only accepts 3d data. seq/6/chars -> seq/6*chars.
x = seqs[:, :-1]
x = x.reshape(x.shape[0], x.shape[1], x.shape[2] * x.shape[3]) #SEQ_LEN/6*chars
y = seqs[:, -1] #6/chars
y = [np.array(y[:, i]) for i in range(6)]

#np.save('data.npy', [x, y]) #most comfortable format to save in, also fastest and most lightweight.

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 16, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 12, 1, 1, 1, 1)


NameError: name 'seqs' is not defined

In [ ]:
from tensorflow.keras import layers


model_input = layers.Input((SEQ_LEN-1, 6*len(char_dict)))

m = layers.LSTM(64, return_sequences=True)(model_input)
m = layers.LSTM(64)(m)
m = layers.Dense(256)(m)
m = layers.LeakyReLU(0.3)(m)
out = layers.Dense(len(char_dict), activation='softmax')(m)
out2 = layers.Dense(len(char_dict), activation='softmax')(m)
out3 = layers.Dense(len(char_dict), activation='softmax')(m)
out4 = layers.Dense(len(char_dict), activation='softmax')(m)
out5 = layers.Dense(len(char_dict), activation='softmax')(m)
out6 = layers.Dense(len(char_dict), activation='softmax')(m)


model = tf.keras.Model(inputs=model_input, outputs=[out,out2,out3,out4,out5,out6], name="tab_generator")
model.summary()


model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(x, y, batch_size=64, epochs=5)#, validation_data=(x_test,y_test))



In [ ]:
# prediction = model.predict(x[16].reshape(1, x.shape[1], x.shape[2]))
# for p in prediction:
#     print(reversed_char_dict[np.argmax(p)])

predictions = []
data = x[191]
for i in range(36):
    prediction = model.predict(data.reshape(1, x.shape[1], x.shape[2])) #prediction shape: (6, 1 ,chars) 6* [[chars]]
    text_prediction = [reversed_char_dict[np.argmax(s)] for s in prediction]
    predictions.append(text_prediction)
    prediction = np.array(prediction)
    prediction = prediction.reshape(1, -1) #(1,6, chars) 
    data[:-1] = data[1:]; data[-1] = prediction

predictions = np.array(predictions)
for i in range(6):
    print(''.join(predictions[:, i]))
        
# for i in range(35): #generate n next words
#     prediction = np.argmax(model.predict(test_data), axis=-1)
    
#     test_data = x[0]
#     test_data[:-1] = test_data[1:];test_data[-1] = prediction[0] #queue-append prediction
#     test_data = test_data.reshape(-1, SEQ_LENGTH)
#     #print(prediction)
#     print(tokenizer.sequences_to_texts([prediction]))

